In [8]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [9]:
def main_task(contamination):
    i_forest = IForest(contamination=contamination)
    i_forest.fit(X_train)
    i_forest_test_pred = i_forest.predict(X_test)
    return round(f1_score(y_test, i_forest_test_pred),3)

In [10]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    contamination = trial.suggest_float('contamination', 0.01, 0.5, step = 0.02)
    print('contamination : ', contamination)
    f1_score = main_task(contamination=contamination)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=100)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 15:03:40,447] A new study created in memory with name: no-name-2387e0ec-dbfe-46ca-8eee-0a5e1509fd23


contamination :  0.17


[I 2023-11-08 15:03:40,969] Trial 0 finished with value: 0.758 and parameters: {'contamination': 0.17}. Best is trial 0 with value: 0.758.


<f1_score: 0.758 >
contamination :  0.23


[I 2023-11-08 15:03:41,228] Trial 1 finished with value: 0.782 and parameters: {'contamination': 0.23}. Best is trial 1 with value: 0.782.


...

<f1_score: 0.783 >
contamination :  0.33


[I 2023-11-08 15:04:08,305] Trial 98 finished with value: 0.778 and parameters: {'contamination': 0.33}. Best is trial 76 with value: 0.807.


<f1_score: 0.778 >
contamination :  0.23


[I 2023-11-08 15:04:08,569] Trial 99 finished with value: 0.779 and parameters: {'contamination': 0.23}. Best is trial 76 with value: 0.807.


<f1_score: 0.779 >
study.best_trial.value : 0.807
study.best_params : {'contamination': 0.25}


In [11]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.807
study.best_paramse : {'contamination': 0.25}
